In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import datetime
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, classification_report
from math import sqrt
from sklearn import linear_model, ensemble, svm, model_selection, metrics
from scipy import stats
import gc

pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_info_columns = 999
pd.options.display.max_info_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_seq_items = 999

%matplotlib inline

In [ ]:
train = pd.read_csv("train.csv")

In [ ]:
test = pd.read_csv("test.csv")

In [ ]:
clusters = pd.read_csv('clusters_6_opt.csv')

In [ ]:
clusters_2 = pd.read_csv('clusters_20_opt.csv')

In [ ]:
clusters_3 = pd.read_csv('clusters_60_opt.csv')

In [ ]:
cols_train = train.columns

In [ ]:
cols1 = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
        '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
        'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b',
        '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992',
        'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd',
        '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a',
        '6619d81fc', '1db387535', 'fc99f9426', '91f701ba2', '0572565c2',
        '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98']

In [ ]:
time_series_imp = cols1[::-1]

In [ ]:
"""
test_compiled_leak = pd.read_csv('test_compiled_leak_36.csv')
augmenation = test_compiled_leak[test_compiled_leak.compiled_leak !=0]
aug = test.merge(augmenation, how='inner', on='ID')
aug['target'] = aug.compiled_leak
aug.drop(['compiled_leak'], inplace=True, axis=1)
X = pd.concat([train,aug], ignore_index=True)
X = X[cols_train]
Y = X['target'].copy()
Y = np.log1p(Y)
X_train = X.drop(['ID','target'],axis=1)
"""

In [ ]:
X = train[cols_train]
Y = X['target'].copy()
Y = np.log1p(Y)
X_train = X.drop(['ID','target'],axis=1)

In [ ]:
train_zeros = pd.DataFrame({'Percent_zero':((X_train.values)==0).mean(axis=0),
                           'Column' : X_train.columns})

high_vol_columns = train_zeros['Column'][train_zeros['Percent_zero'] < 0.76].values
low_vol_columns = train_zeros['Column'][train_zeros['Percent_zero'] >= 0.76].values

low_vol_columns_to_drop = train_zeros['Column'][train_zeros['Percent_zero'] >= 0.62].values
high_vol_columns_to_drop = train_zeros['Column'][train_zeros['Percent_zero'] < 0.62].values

low_vol_columns_to_drop_1 = train_zeros['Column'][train_zeros['Percent_zero'] >= 0.76].values

In [ ]:
def statsForHighLow(X1, df):
    cluster_sets = {"low":low_vol_columns, "high":high_vol_columns}
    for cluster_key in cluster_sets:
        X1["count_not0_"+cluster_key] = df[cluster_sets[cluster_key]].count(axis=1)
        X1["sum_"+cluster_key] = df[cluster_sets[cluster_key]].sum(axis=1)
        X1["var_"+cluster_key] = df[cluster_sets[cluster_key]].var(axis=1)
        X1["median_"+cluster_key] = df[cluster_sets[cluster_key]].median(axis=1)
        X1["mean_"+cluster_key] = df[cluster_sets[cluster_key]].mean(axis=1)
        X1["std_"+cluster_key] = df[cluster_sets[cluster_key]].std(axis=1)
        X1["max_"+cluster_key] = df[cluster_sets[cluster_key]].max(axis=1)
        X1["min_"+cluster_key] = df[cluster_sets[cluster_key]].min(axis=1)
        X1["skew_"+cluster_key] = df[cluster_sets[cluster_key]].skew(axis=1)
        X1["kurtosis_"+cluster_key] = df[cluster_sets[cluster_key]].kurtosis(axis=1) 
    return X1

In [ ]:
def statsForClusters(X1, tmp, clusters, k):
    for j in range(k):
        try:
            cols = clusters[clusters.cluster == j].feature.values
            X1['mean_nonz_'+str(j)] = tmp[cols].mean(axis=1)
            X1['std_nonz_'+str(j)] = tmp[cols].std(axis=1)
            X1['median_nonz_'+str(j)] = tmp[cols].median(axis=1)
            X1['range_'+str(j)] = tmp[cols].max(axis=1) - tmp[cols].min(axis=1)
            X1['coef_var_nonz_'+str(j)] = (X1['std_nonz_'+str(j)]/ X1['mean_nonz_'+str(j)])*100
            msk1 = np.ma.masked_invalid(tmp[cols])
            X1['gmean_'+str(j)] = stats.mstats.gmean(msk1, axis=1)
            X1['gmean/mean_'+str(j)] = X1['gmean_'+str(j)]/X1['mean_nonz_'+str(j)]
            X1['count_nonz_'+str(j)] = tmp[cols].notnull().sum(axis=1)
            X1['count_unique_'+str(j)] = tmp[cols].T.apply(lambda x: len(x.unique())-1)
            mode_nonzero = stats.mode(tmp[cols], axis=1, nan_policy='omit')
            X1['frequent_of_mode'+str(j)] = mode_nonzero[1].data
            X1['mode_nonzero'+str(j)] = mode_nonzero[0].data
            X1['mode_for_freq'+str(j)] = X1[X1['frequent_of_mode'+str(j)] > 3]['mode_nonzero'+str(j)]
            X1.drop(['mode_nonzero'+str(j)], axis=1, inplace=True)
        except:
            pass
    return X1

In [ ]:
def preparingInput(df):
    X1 = df.copy()
    X1[X1 == 0] = np.nan
    X1['mean'] = df.mean(axis=1)
    X1['std'] = df.std(axis=1)
    X1['var'] = df.var(axis=1)
    X1['count_nonzero'] = np.count_nonzero(df, axis=1)
    X1['unique/nonzero'] = df.T.apply(lambda x: len(x.unique())-1)/X1['count_nonzero']
    X1['sum'] = df.sum(axis=1)
    X1['mean_nonzero'] = X1['sum']/X1['count_nonzero']
    X1['mean/mean_nonzero'] = X1['mean']/X1['mean_nonzero']
    X1['kurtosis'] = stats.kurtosis(df, axis=1)
    X1['skew'] = stats.skew(df, axis=1)
    msk = np.ma.masked_where(df == 0, df)
    X1['median_nonzero'] = np.ma.median(msk, axis=1)
    X1['gmean'] = stats.mstats.gmean(msk, axis=1)
    X1['min_nonzero'] = np.ma.min(msk, axis=1)
    X1['std_nonzero'] = np.ma.std(msk, axis=1)
    X1['coef_var_nonzero'] = (X1['std_nonzero']/ X1['mean_nonzero'])*100
    X1['interquartile_range'] = stats.mstats.mquantiles(msk, prob=[0.75], axis=1) - stats.mstats.mquantiles(msk, prob=[0.25], axis=1)
    X1['kurtosis_nonzero'] = stats.mstats.kurtosis(msk, fisher=False, axis=1)
    X1['skewness_nonzero'] = stats.mstats.skew(msk, axis=1)
    X1['gmean_mean_nonzero'] = X1['gmean']/X1['mean_nonzero']
    X1['gmean_mean'] = X1['gmean']/X1['mean']
    X1['inverse_gmean_mean_nonzero'] = X1['mean_nonzero']/X1['gmean']
    X1['percentile_90'] = np.percentile(df, 90, axis=1)
    X1['percentile_91'] = np.percentile(df, 91, axis=1)
    X1['percentile_92'] = np.percentile(df, 92, axis=1)
    X1['percentile_93'] = np.percentile(df, 93, axis=1)
    X1['percentile_94'] = np.percentile(df, 94, axis=1)
    X1['percentile_95'] = np.percentile(df, 95, axis=1)
    X1['percentile_96'] = np.percentile(df, 96, axis=1)
    X1['percentile_97'] = np.percentile(df, 97, axis=1)
    X1['percentile_98'] = np.percentile(df, 98, axis=1)
    X1['percentile_99'] = np.percentile(df, 99, axis=1)
    X1['max'] = df.max(axis=1)
    tmp = df.copy()
    tmp[tmp == 0] = np.nan
    mode_nonzero = stats.mode(tmp, axis=1, nan_policy='omit')
    X1['frequent_of_mode'] = mode_nonzero[1].data
    X1['mode_nonzero'] = mode_nonzero[0].data
    X1['mode_for_freq'] = X1[X1.frequent_of_mode > 5].mode_nonzero
    X1.drop(['mode_nonzero'], axis=1, inplace=True)
    statsForClusters(X1, tmp, clusters, 6)
    statsForHighLow(X1, tmp)
    X2 = X1.drop(low_vol_columns_to_drop,axis=1)
    X3 = tmp.copy()
    msk2 = np.ma.masked_invalid(tmp[time_series_imp])
    statsForClusters(X3, tmp, clusters, 6)
    statsForHighLow(X3, tmp)
    X4 = X3.drop(low_vol_columns_to_drop,axis=1)
    X5 = X3.drop(high_vol_columns_to_drop,axis=1)
    X6 = tmp.copy().drop(low_vol_columns_to_drop_1,axis=1)
    statsForHighLow(X6, tmp)
    X7 = X4.copy()
    X8 = tmp[cols1].copy()
    X9 = pd.DataFrame()
    X7['last_nonz_index'] = pd.Series(np.ma.notmasked_edges(msk2, axis=1)[1][1])
    X7['last_nonz'] = tmp[time_series_imp].ewm(alpha=1, ignore_na = True, axis=1).mean().iloc[:,-1]
    X7['last_nonz_2'] = tmp[time_series_imp].ewm(alpha=1, ignore_na = True, axis=1).mean().iloc[:,-2]
    X7['last_nonz_3'] = tmp[time_series_imp].ewm(alpha=1, ignore_na = True, axis=1).mean().iloc[:,-3]
    X7['1_diff'] = X7['last_nonz'] - X7['last_nonz_2']
    statsForClusters(X8, tmp, clusters_2, 20)
    statsForClusters(X9, tmp, clusters_3, 60)
    X2 = X2.loc[:, X2.std() != 0.0]
    X4 = X4.loc[:, X4.std() != 0.0]
    X5 = X5.loc[:, X5.std() != 0.0]
    X6 = X6.loc[:, X6.std() != 0.0]
    X7 = X7.loc[:, X7.std() != 0.0]
    X8 = X8.loc[:, X8.std() != 0.0]
    X9 = X9.loc[:, X9.std() != 0.0]
    return [X2, X4, X5, X7, X8, X6, X9]
    

In [ ]:
del train
gc.collect()

In [ ]:
input_for_model = preparingInput(X_train)

In [ ]:
gc.collect()

In [ ]:
X1 = input_for_model[0] #
X2 = input_for_model[1]
X3 = input_for_model[2]
X4 = input_for_model[3]
X5 = input_for_model[4]
X6 = input_for_model[5]
X7 = input_for_model[6]

In [ ]:
X6.shape

### The model

In [ ]:
parameters_2 = {
    'application': 'regression',
    'metric': 'l2_root',
    'boosting': 'gbdt',
    'num_leaves': 150,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'learning_rate': 0.005, 
    'verbose': 50
}

In [ ]:
parameters_1 = {
    'application': 'regression',
    'metric': 'l2_root',
    'boosting': 'gbdt',
    'num_leaves': 120,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'learning_rate': 0.01, 
    'verbose': 100
}

In [ ]:
X2.head()

In [ ]:
def trainModel(X, Y, parameters):
    cv = KFold(n_splits=5, shuffle=True, random_state=1)
    i=1
    results = pd.DataFrame()
    y_pred_test_oof = pd.DataFrame({'Y_oof': np.zeros(Y.shape)})
    models = list()
    for trainIdx, testIdx in cv.split(X, Y):
        train_data = lgb.Dataset(X.values[trainIdx], label=Y.values[trainIdx])
        test_data = lgb.Dataset(X.values[testIdx], label=Y.values[testIdx])
        model = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)
        y_pred_train = model.predict(X.values[trainIdx])
        y_pred_test = model.predict(X.values[testIdx])
        y_pred_test_oof.iloc[testIdx, 0] =  y_pred_test
        models.append(model)
        results.loc[i, 'train_RMSE'] = sqrt(mean_squared_error(y_pred_train, Y.values[trainIdx]))
        results.loc[i, 'test_RMSE'] = sqrt(mean_squared_error(y_pred_test, Y.values[testIdx]))
        i+=1
    return [results, y_pred_test_oof, models]

In [ ]:
%%time
training_2 = trainModel(X2, Y, parameters_1)

In [ ]:
results = training_2[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_2[1], Y)))

In [ ]:
gc.collect()

In [ ]:
%%time
training_1 = trainModel(X1, Y, parameters_2)

In [ ]:
results = training_1[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_1[1], Y)))

In [ ]:
gc.collect()

In [ ]:
%%time
training_3 = trainModel(X3, Y, parameters_2)

In [ ]:
results = training_3[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_3[1], Y)))

In [ ]:
gc.collect()

In [ ]:
%%time
training_4 = trainModel(X4, Y, parameters_1)

In [ ]:
results = training_4[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_4[1], Y)))

In [ ]:
%%time
training_5 = trainModel(X5, Y, parameters_1)

In [ ]:
results = training_5[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_5[1], Y)))

In [ ]:
%%time
training_6 = trainModel(X6, Y, parameters_1)

In [ ]:
results = training_6[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_6[1], Y)))

In [ ]:
%%time
training_7 = trainModel(X7, Y, parameters_2)

In [ ]:
results = training_7[0]
print(results)
print("Mean: {:4f}".format(np.mean(results.test_RMSE)))
print("Standart deviation: {:4f}".format(np.std(results.test_RMSE)))
print("OOF-rmse: ", sqrt(mean_squared_error(training_7[1], Y)))

In [ ]:
sqrt(mean_squared_error((0.2*training_1[1] + 0*training_2[1]  +0.07*training_3[1]+ 0.2*training_4[1]+0.15*training_5[1]+0.18*training_6[1] + 0.2*training_7[1]), Y))


In [ ]:
del input_for_model, X1, X2, X3, X4, X5, X6, X7
gc.collect()

### Prediction

In [ ]:
input_test = preparingInput(test[cols_train.drop(['ID', 'target'])])

In [ ]:
training = [training_1, training_2, training_3, training_4, training_5, training_6, training_7]

In [ ]:
my_predictions = []

In [ ]:
k=0
for j in training:
    my_prediction = pd.DataFrame()
    i=0
    for i in range(5):
        my_prediction[str(i)] = pd.Series(j[2][i].predict(input_test[k], num_iteration = j[2][i].best_iteration))
        i+=1
    my_predictions.append(pd.Series(my_prediction.mean(axis=1)))
    k+=1

In [ ]:
predict = 0.2*my_predictions[0] + 0.0*my_predictions[1]+0.1*my_predictions[2]+ 0.2*my_predictions[3]+0.15*my_predictions[4]+0.15*my_predictions[5] + 0.2*my_predictions[6]


In [ ]:
my_pred = np.expm1(predict)

In [ ]:
my_pred = np.expm1(my_predictions[1])

In [ ]:
len(my_pred)

In [ ]:
my_submission = pd.DataFrame(test[['ID']]).reset_index().join(pd.DataFrame(my_pred, columns=['target']))

In [ ]:
my_submission = my_submission.drop(['index'], axis=1)

In [ ]:
my_submission

In [ ]:
my_submission.to_csv('my_submission_new!_new!_new.csv', index=False)